In [3]:
import py_trees
import random
import time
from py_trees import Blackboard
import bt_utils

In [4]:
class TurnTowardsWall(bt_utils.BaseBehavior):
    def update(self):
        bb = Blackboard()
        if bb.angle == 0:
            self.log("Now pointing to wall")
            bb.facing = True
            return (py_trees.Status.SUCCESS)
        else:
            bb.angle -= 10
            self.log("Turning towards wall",bb.angle)
            return(py_trees.Status.RUNNING)

In [5]:
class DriveToWall(bt_utils.BaseBehavior):
    def update(self):
        bb = Blackboard()
        if not bb.facing:
            return py_trees.Status.FAILURE
        if bb.distance > 5:
            self.log("Not near wall yet")
            return py_trees.Status.RUNNING
        else:
            self.log("near wall")
            return py_trees.Status.SUCCESS

In [6]:
class TooClose(bt_utils.BaseBehavior):
    def update(self):
        bb = Blackboard()
        if bb.distance < 5:
            self.log("Too Close to wall!")
            return py_trees.Status.FAILURE
        else:
            self.log("Now at a safe distance, not to worry")
            return py_trees.Status.SUCCESS

In [7]:
class MoveAwayFromWall(bt_utils.BaseBehavior):
    def update(self):
        bb = Blackboard()
        if bb.distance < 10:
            self.log("Emergency pulling away")
            bb.distance += 1
            return py_trees.Status.RUNNING
        else:
            self.log("Enough distance. Emergency over")
            return py_trees.Status.SUCCESS

In [12]:
def create_tree():
    bb = Blackboard()
    bb.facing = False
    bb.distance = 2
    bb.angle = 30
    range_safety = py_trees.composites.Selector("rangsaf")
    range_safety.add_child(TooClose("tooclose"))
    range_safety.add_child(MoveAwayFromWall("awaywall"))
    seq = py_trees.composites.Sequence(name="orient")
    seq.add_child(range_safety)
    seq.add_child(TurnTowardsWall("turntowall"))
    seq.add_child(DriveToWall("drivetowall"))
    return seq

In [13]:
tree = create_tree()
tree.setup(timeout=15)
bb = Blackboard()
bb.counter = 1
while tree.status != py_trees.Status.SUCCESS:
    try:
        print("--------- Tick {0} ---------".format(bb.counter))
        tree.tick_once()
        py_trees.display.print_ascii_tree(tree, show_status=True)
        time.sleep(1.0)
        bb.counter += 1
        print(bb.angle, bb.distance, bb.facing)
    except KeyboardInterrupt:
        break


tooclose:  __init__
awaywall:  __init__
turntowall:  __init__
drivetowall:  __init__
tooclose:  setup
awaywall:  setup
turntowall:  setup
drivetowall:  setup
--------- Tick 1 ---------
tooclose:  Too Close to wall!
awaywall:  Emergency pulling away
orient [*]
(-) rangsaf [*]
    --> tooclose [✕]
    --> awaywall [*]
--> turntowall [-]
--> drivetowall [-]
(30, 3, False)
--------- Tick 2 ---------
tooclose:  Too Close to wall!
awaywall:  Emergency pulling away
orient [*]
(-) rangsaf [*]
    --> tooclose [✕]
    --> awaywall [*]
--> turntowall [-]
--> drivetowall [-]
(30, 4, False)
--------- Tick 3 ---------
tooclose:  Too Close to wall!
awaywall:  Emergency pulling away
orient [*]
(-) rangsaf [*]
    --> tooclose [✕]
    --> awaywall [*]
--> turntowall [-]
--> drivetowall [-]
(30, 5, False)
--------- Tick 4 ---------
tooclose:  Now at a safe distance, not to worry
turntowall: Turning towards wall 20
orient [*]
(-) rangsaf [✓]
    --> tooclose [✓]
    --> awaywall [-]
--> turntowall [*]
-